In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [ ]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

_requests = [
    LLMRequest(
        conversation=LLMConversation(
            messages=[
                LLMMessage(role='system', content='You are a helpful assistant.'),
                LLMMessage(role='user', content=f'what is {i}+2'),
            ]
        ),
        params=LLMParams(
            model=MODEL_HUB_ID,
            temperature=0,
            response_type=SomeOutput,
            max_completion_tokens=10,
            metadata={'input_id': str(some_input.id)},
        ),
    )
    for i in range(50)
]

batch_request: LLMBatchRequest = LLMBatchRequest(requests=_requests)

In [5]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

In [5]:
from math_rag.application.models.inference import (
    EMBatchRequest,
    EMParams,
    EMRequest,
)


MODEL_HUB_ID = 'BAAI/bge-large-en-v1.5'

_em_requests = [
    EMRequest(
        text=f'hello world {i}',
        params=EMParams(model=MODEL_HUB_ID, dimensions=1024),
    )
    for i in range(50)
]

em_batch_request: EMBatchRequest = EMBatchRequest(requests=_em_requests)

### Apptainer

In [6]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

tgi_batch_llm = infrastructure_container.tgi_batch_llm()
tei_batch_em = infrastructure_container.tei_batch_em()

In [ ]:
await tgi_batch_llm.init_resources()

In [ ]:
res = await tgi_batch_llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

In [7]:
await tei_batch_em.init_resources()

2025-05-09 20:08:43,685 - INFO - Host canonicalization disabled
2025-05-09 20:08:43,686 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-09 20:08:43,720 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-09 20:08:43,720 - INFO - [conn=0]   Local address: 172.18.0.5, port 48770
2025-05-09 20:08:43,721 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-05-09 20:08:43,851 - INFO - [conn=0] Beginning auth for user lpanic
2025-05-09 20:08:44,148 - INFO - [conn=0] Auth for user lpanic succeeded
2025-05-09 20:08:44,151 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-05-09 20:08:44,198 - INFO - [conn=0, chan=0]   Command: mkdir -p tei_default_root
2025-05-09 20:08:44,361 - INFO - [conn=0, chan=0] Received exit status 0
2025-05-09 20:08:44,361 - INFO - [conn=0, chan=0] Received channel close
2025-05-09 20:08:44,362 - INFO - [conn=0, chan=0] Channel closed
2025-05-09 20:08:44,362 - INFO - Command `mkdir -p tei_default_r

In [ ]:
res = await tei_batch_em.batch_embed(
    batch_request=em_batch_request,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

2025-05-09 20:15:50,917 - INFO - Host canonicalization disabled
2025-05-09 20:15:50,918 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-05-09 20:15:50,931 - INFO - [conn=25] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-05-09 20:15:50,932 - INFO - [conn=25]   Local address: 172.18.0.5, port 53860
2025-05-09 20:15:50,932 - INFO - [conn=25]   Peer address: 161.53.2.37, port 22
2025-05-09 20:15:50,971 - INFO - [conn=25] Beginning auth for user lpanic
2025-05-09 20:15:51,238 - INFO - [conn=25] Auth for user lpanic succeeded
2025-05-09 20:15:51,241 - INFO - [conn=25, chan=0] Requesting new SSH session
2025-05-09 20:15:51,279 - INFO - [conn=25, chan=0]   Subsystem: sftp
2025-05-09 20:15:51,289 - INFO - [conn=25, chan=0] Starting SFTP client
2025-05-09 20:15:51,509 - INFO - [conn=25, chan=0] SFTP client exited
2025-05-09 20:15:51,510 - INFO - [conn=25, chan=0] Closing channel
2025-05-09 20:15:51,512 - INFO - [conn=25, chan=0] Received exit status 0
2025

In [ ]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'

%run ../assets/hpc/hf/tgi/tgi_client.py

In [ ]:
import asyncio
import threading


# NOTE: prometheus_snapshot_loader_service.load will be run as a background task in fast api
def run_async_in_thread(coro):
    def runner():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(coro)
        loop.close()

    thread = threading.Thread(target=runner)
    thread.start()

    return thread


prometheus_snapshot_loader_service = (
    infrastructure_container.prometheus_snapshot_loader_service()
)
run_async_in_thread(prometheus_snapshot_loader_service.load())